<div style="text-align: right"> Provided on April 1 and due on April 15 [BRI516, Spring/2020] </div>

For homework in general:
* Install `Anaconda` and create a python environment with `NumPy`, `Pandas`, `Matplotlib`, `scikit-learn` in Python >= 3.5 
* Visualize the results of the code as much as possible and add comments/description on the code/results
* Please upload your jupyter-notebook file for homework to `Blackboard`

### [HW#1] Perceptron, logistic regression, and support vector machine

1. Load the Iris data ('iris.csv') using Pandas


2. Divide the overall samples (n=150) into 80% for training and 20% for test

(a) Using the scikit-learn's `train_test_split` class with a `stratify` option

(b) Implement your own class, `my_train_test_split` with a `stratify` option available

(c) Compare the results from (a) and (b) 


<br>

※ In 3~5, either is fine to use train/test sets from 'train_test_split' or 'my_train_test_split'


3. Apply the Perceptron model to classfy the three flower classes as good as possible

(a) Using the class, `Perceptron` in scikit-learn 

(b) Using the class, `SGDClassifier` in scikit-learn: implement stochastic gradient algorithm of the Perceptron using the `partial_fit` method of this class for a varying mini-batch sizes (i.e., mini-batch size = 1, 2, 10)

(c) Discuss the results from (a) and (b)


<br>

4. Apply the logistic regression (LR) model to classfy the three flower classes as good as possible

(a) Using the class, `LogisticRegression` in scikit-learn 


(b) Using the class, `SGDClassifier` in scikit-learn: implement stochastic gradient algorithm of the Perceptron using the `partial_fit` method of this class for a varying mini-batch sizes (i.e., mini-batch size = 1, 2, 10)

(c) Discuss the results from (a) and (b)


<br>

5. Apply the support vector machine (SVM) model to classfy the three flower classes as good as possible

(a) Using the class, `SVC` in scikit-learn 


(b) Using the class, `SGDClassifier` in scikit-learn: implement stochastic gradient algorithm of the Perceptron using the `partial_fit` method of this class for a varying mini-batch sizes (i.e., mini-batch size = 1, 2, 10)

(c) Discuss the results from (a) and (b)


<br>

6. Discuss the results from (3) to (5)

Notes:
* Students are supposed to set hyperparams (such as number of epochs) on their own
* For each of the classification tests, show the following: <br>
(a) learning curves (along epochs) of the cost/error from the training and test data where applicable <br>
(b) learning curves (along epochs) of average weight update term for each epoch (i.e., $\Delta w$) <br>
(c) decision boundary of the trained model along with training and test samples in the scatter plot

* Draw plots using matplotlib

* Please discuss each of your results with at least few lines of text